In [1]:
import os
import pandas as pd

In [2]:
train_df = pd.read_csv('./train.csv')

In [3]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test_df = pd.read_csv('./test.csv')
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
test_df[~test_df['keyword'].isna()]

,id,keyword,location,text
15,46,ablaze,London,Birmingham Wholesale Market is ablaze BBC News...
16,47,ablaze,Niall's place | SAF 12 SQUAD |,@sunkxssedharry will you wear shorts for race ...
17,51,ablaze,NIGERIA,#PreviouslyOnDoyinTv: Toke MakinwaÛªs marriag...
18,58,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...
19,60,ablaze,"Los Angeles, Califnordia",PSA: IÛªm splitting my personalities.\n\n?? t...
...,...,...,...,...
3247,10806,wrecked,Seattle Washington,RT CNBC '3 words from Disney CEO Bob Iger wrec...
3248,10807,wrecked,Acey mountain islanddåÇTorontoåÈ,Smackdown tyme this should put me in a good mo...
3249,10816,wrecked,los angeles,@thrillhho jsyk I haven't stopped thinking abt...
3250,10820,wrecked,"Brussels, Belgium",@stighefootball Begovic has been garbage. He g...


In [6]:
test_df[~test_df['location'].isna()]

,id,keyword,location,text
15,46,ablaze,London,Birmingham Wholesale Market is ablaze BBC News...
16,47,ablaze,Niall's place | SAF 12 SQUAD |,@sunkxssedharry will you wear shorts for race ...
17,51,ablaze,NIGERIA,#PreviouslyOnDoyinTv: Toke MakinwaÛªs marriag...
18,58,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...
19,60,ablaze,"Los Angeles, Califnordia",PSA: IÛªm splitting my personalities.\n\n?? t...
...,...,...,...,...
3246,10804,wrecked,Love Reiss,@yakubOObs think he deactivated because his no...
3247,10806,wrecked,Seattle Washington,RT CNBC '3 words from Disney CEO Bob Iger wrec...
3248,10807,wrecked,Acey mountain islanddåÇTorontoåÈ,Smackdown tyme this should put me in a good mo...
3249,10816,wrecked,los angeles,@thrillhho jsyk I haven't stopped thinking abt...


In [7]:
import re
def create_text_feature(df):
    df = df.fillna('unknown')
    res = df['keyword'].apply(lambda r: f"keyword {r}") + \
        df['location'].apply(lambda r: f", location {r}, ") + \
        df['text'].apply(lambda r: re.sub(r'^https?:\/\/.*[\r\n]*', '', r, flags=re.MULTILINE))
    return res

# ORYGINAL VERSION:
#def create_text_feature(df):
#    df = df.fillna('')
#    res = df['keyword'].apply(lambda r: f"keyword: [{r}]") + \
#        df['location'].apply(lambda r: f", location: [{r}], ") + \
#        df['text']
#    return res



In [8]:
create_text_feature(test_df).head()

0    keyword unknown, location unknown, Just happen...
1    keyword unknown, location unknown, Heard about...
2    keyword unknown, location unknown, there is a ...
3    keyword unknown, location unknown, Apocalypse ...
4    keyword unknown, location unknown, Typhoon Sou...
dtype: object

In [9]:
train_df.fillna('unknown', inplace=True)
train_df['all_text'] = create_text_feature(train_df)
train_df['target'] = train_df['target'].apply(lambda v: str(v))

In [10]:
train_val_df = train_df.sample(n=512, random_state=101)
train_tra_df = train_df[~train_df.isin(train_val_df)].dropna()

In [11]:
train_tra_df[['target', 'all_text']].rename(columns = {'all_text': 'text'}).to_csv('./train_formatted.csv', index=False)

In [12]:
train_val_df[['target', 'all_text']].rename(columns = {'all_text': 'text'}).to_csv('./valid_formatted.csv', index=False)

In [13]:
test_df['all_text'] = create_text_feature(test_df)
test_df['target'] = 0

In [14]:
test_df[['target', 'all_text']].rename(columns = {'all_text': 'text'}).to_csv('./test_formatted.csv', index=False)

In [15]:
print(len(train_df), len(train_val_df) + len(train_tra_df), len(train_val_df), len(train_tra_df))
print(len(test_df))

7613 7613 512 7101
3263
